In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.feature_extraction import DictVectorizer

In [77]:
import pandas as pd
data_train = pd.read_csv("salary-train.csv")
data_test = pd.read_csv("salary-test-mini.csv")
data_train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [89]:
data_test

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,we currently have a vacancy for an hr project ...,Milton Keynes,contract,NaN
1,a web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [17]:
data_test.LocationNormalized.unique()

array(['Milton Keynes', 'Manchester'], dtype=object)

In [79]:
data_train['LocationNormalized'].fillna('nan', inplace=True)
data_train['ContractTime'].fillna('nan', inplace=True)

In [85]:
data_train['LocationNormalized'].isna().sum()

0

In [83]:
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [84]:
import re
data_train['FullDescription'] = data_train.FullDescription.apply(lambda text: re.sub("[^a-zA-Z0-9]", ' ',  text.lower()))
data_test['FullDescription'] = data_test.FullDescription.apply(lambda text: re.sub("[^a-zA-Z0-9]", ' ',  text.lower()))

In [86]:
vectorizer = TfidfVectorizer(min_df=5)
FullDescription_train = vectorizer.fit_transform(data_train['FullDescription'])
FullDescription_test = vectorizer.transform(data_test['FullDescription'])

In [66]:
FullDescription_train

<60000x22861 sparse matrix of type '<class 'numpy.float64'>'
	with 8365759 stored elements in Compressed Sparse Row format>

In [87]:
import scipy
X_train = scipy.sparse.hstack([FullDescription_train, X_train_categ])
X_test = scipy.sparse.hstack([FullDescription_test, X_test_categ])

In [93]:
X_train

<60000x24627 sparse matrix of type '<class 'numpy.float64'>'
	with 8485759 stored elements in COOrdinate format>

In [88]:
clf = Ridge(alpha=1.0)
y = data_train['SalaryNormalized']
clf.fit(X_train, y)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [92]:
ans = list(map(lambda x: round(x,2), clf.predict(X_test)))
def write2file_many(num1, num2, filename):
    with open(filename, 'w') as f:
        f.write(f"{num1} {num2}")
write2file_many(ans[0], ans[1], '1.txt')